In [61]:
import json
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from tqdm import tqdm
import numpy as np
import scann
from IPython.display import HTML,display

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
result_df = pd.read_csv('../../examples/relevant_jobs_classification/data/score_result_v1.csv')

In [78]:
pd.DataFrame(result_df['job_id'].unique(),columns=['job_id'])

,job_id
0,1373732
1,1373755
2,1373879
3,1373929
4,1374956
5,1374981
6,1375026
7,1375642
8,1376033


In [3]:
import requests

def fetch_jobs(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    

    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    # print(response.text)
    

    # url = url.replace("<DAYS>",str(last_k)).replace("<SIZE>",str(size))

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()
    # print(job_data)
    job_df = pd.DataFrame(job_data)

    return job_df[['id','title']]

In [4]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    # result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        # 'keywords': [],
       'current_designation' : [],
        'user_experience':[],  
    
    }
    
    for s in tqdm(user_json):
    
        try:
            source = s[0]
    
            user_dict['id'].append(source['id'])
    
            # skill_list = [str(skill['name']).replace(" ","_") for skill in source['skillSet']]
            # user_dict['keywords'].append(" ".join(skill_list))
            
            user_dict['user_experience'].append(source['expYear'])
            
            # user_dict['resume'].append(source['resumeText'])
            user_dict['current_designation'].append(source['currentDesignation'])
    
        except Exception as e:
            continue
    
    return user_dict

In [24]:
def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

def get_resume_link(userid_list):

    base_str = "https://search.iimjobs.com/profile/<USERID>"
    resume_links = [base_str.replace('<USERID>',str(user_id)) for user_id in userid_list]
    
    return resume_links

,id,title
0,1373732,Renaissance Global - Equity Research Analyst -...
0,1373755,Manager - Talent Acquisition - BPO (8-12 yrs)
0,1373879,Senior Manager - Business Operations - Real Es...
0,1373929,ROL Agri - Area Sales Manager - Agrochemical -...
0,1374956,Director - Education & Government Consulting (...
0,1374981,Regional Sales Head - LAP Business (5-12 yrs)
0,1375026,Cluster Manager - Retail (5-15 yrs)
0,1375642,Finance Manager/Finance Controller - CA (2-8 yrs)
0,1376033,Lal PathLabs - Assistance General Manager - S...


In [25]:
get_resume_link([2700303,1350766])

['https://search.iimjobs.com/profile/2700303',
 'https://search.iimjobs.com/profile/1350766']

In [26]:

def get_user_data_in_batches(uid_list, batch_size=1000):
    user_data = pd.DataFrame()

    for i in range(0, len(uid_list), batch_size):
        batch_uids = uid_list[i:i + batch_size]

        batch_user_data = get_user_data_search_embed(batch_uids)

        user_data = pd.concat([user_data, batch_user_data], ignore_index=True)

    return user_data

In [27]:
# def clean_text(input_text):
    
#     '''Make text lowercase, remove text in square brackets,remove links,
#     remove punctuation and remove words containing numbers.'''


#     text = input_text
        
#     text = str(text).lower()
#     text = re.sub('\[.*?\]',' ', text)
#     text = re.sub('https?://\S+|www\.\S+',' ', text)
#     text = re.sub('<.*?>+',' ', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation),' ', text)
#     text = re.sub('\n',' ', text)
#     text = re.sub('\w*\d\w*',' ', text)
#     text = re.sub('yrs',' ',text)
#     text = re.sub('\r',' ',text)
#     text = ' '.join([word for word in text.split() if word not in (stop)])
#     text = re.sub(' +', ' ', text)
    
#     return text

In [28]:
# li = []
# for d in tqdm(main_df['introText']):
#     des = clean_text(d)
#     li.append(des)

In [29]:
# main_df['description']=li

In [30]:
# 

In [84]:
id_list = [fetch_jobs(str(k)) for k in result_df['job_id'].unique().tolist()]
pd.concat(id_list)

,id,title
0,1373732,Renaissance Global - Equity Research Analyst -...
0,1373755,Manager - Talent Acquisition - BPO (8-12 yrs)
0,1373879,Senior Manager - Business Operations - Real Es...
0,1373929,ROL Agri - Area Sales Manager - Agrochemical -...
0,1374956,Director - Education & Government Consulting (...
0,1374981,Regional Sales Head - LAP Business (5-12 yrs)
0,1375026,Cluster Manager - Retail (5-15 yrs)
0,1375642,Finance Manager/Finance Controller - CA (2-8 yrs)
0,1376033,Lal PathLabs - Assistance General Manager - S...


In [87]:
def do_something(jid):
    # jid = result_df.job_id.sample().unique()[0]
    # print(jid)
    r = result_df.loc[result_df['job_id']==jid]
    job_df = fetch_jobs(str(jid))
    uid_list = list(result_df.loc[result_df['job_id']==jid].user_id)
    user_df = get_user_data_in_batches(uid_list)
    user_df['id']=user_df['id'].astype(int)
    r = r[['user_id','scores']]
    r.columns = ['id','scores']
    r['id']=r['id'].astype(int)
    final_df = pd.merge(user_df,r,on='id',how='left')
    final_df.sort_values(['scores'],ascending=False, inplace=True)

    final_df['resume_links'] = get_resume_link(final_df['id'])
    final_df['resume_links'] = final_df['resume_links'].apply(make_clickable)
    
    return job_df,final_df

In [91]:
a,b= do_something(1374981)
display(a,HTML(b.to_html(escape=False)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 565319.23it/s]


,id,title
0,1374981,Regional Sales Head - LAP Business (5-12 yrs)


,id,current_designation,user_experience,scores,resume_links
18,244673,Director,9,0.638066,https://search.iimjobs.com/profile/244673
23,443888,"Regional Head - East Branch Asset, Branch Asset Desk",17,0.627340,https://search.iimjobs.com/profile/443888
27,1959483,Assistant General Manager,8,0.618618,https://search.iimjobs.com/profile/1959483
0,937279,Deputy General Manager,8,0.614850,https://search.iimjobs.com/profile/937279
1,1263900,Regional business Head,14,0.610414,https://search.iimjobs.com/profile/1263900
3,1918487,Regional Manager,6,0.608794,https://search.iimjobs.com/profile/1918487
26,1661525,Sr Inside sales leader,1,0.580605,https://search.iimjobs.com/profile/1661525
25,238941,"Regional Business Head, North (All Categories) - Jiomart",10,0.576496,https://search.iimjobs.com/profile/238941
8,1196651,Location Head- Education Loan,15,0.565705,https://search.iimjobs.com/profile/1196651
15,1268621,Project Manager,5,0.562047,https://search.iimjobs.com/profile/1268621


In [21]:
# d

In [34]:
# list(result_df.loc[result_df['job_id']==jid].user_id)